In [1]:
%%capture
!pip install spacy
!python -m spacy download en_core_web_lg


In [33]:
import nltk

from string import punctuation
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
import spacy
import en_core_web_lg
nlp = en_core_web_lg.load()


/usr/local/lib/python3.7/dist-packages/spacy/language.py:1899: UserWarning: [W123] Argument disable with value [] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  config_value=config["nlp"][key],


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pandas as pd
scores = pd.read_csv('drive/My Drive/similarity_scores.csv')

In [7]:
scores.drop('similarity_score', axis=1, inplace=True)


In [8]:
scores

,Unnamed: 0,product_name,product_review
0,1,Kentucky Brunch Brand Stout,I didnt think i was going to give it a perfect...
1,2,Kentucky Brunch Brand Stout,So I just read a review that called the legend...
2,3,Kentucky Brunch Brand Stout,2021 vintage bottle 79\n\nHoly Fucking Shit KB...
3,4,Kentucky Brunch Brand Stout,Celebrating my buddy Rug with his 1000th beer ...
4,5,Kentucky Brunch Brand Stout,Thick and syrupy pour mocha head Aroma is booz...
...,...,...,...
2508,2509,Coolship Red,Very interesting and complex ale First this is...
2509,2510,Coolship Red,Appearance The beer was pinkish red is color ...
2510,2511,Coolship Red,Look beautiful opaque garnet red Soft white he...
2511,2512,Coolship Red,Caged and corked bottle dated dec32020 and ope...


In [16]:
import re
from collections import Counter


WORD = re.compile(r"\w+")


In [17]:
def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)


In [18]:
attr = 'chocolate dark sweet bourbon coffee'
attr_vector = text_to_vector(attr)


In [19]:
doc1 = nlp(attr)

In [23]:
scores['spacy_score'] = scores['product_review'].apply(lambda doc2: doc1.similarity(nlp(doc2)))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  """Entry point for launching an IPython kernel.


In [24]:
scores

,Unnamed: 0,product_name,product_review,spacy_score
0,1,Kentucky Brunch Brand Stout,I didnt think i was going to give it a perfect...,0.488775
1,2,Kentucky Brunch Brand Stout,So I just read a review that called the legend...,0.454404
2,3,Kentucky Brunch Brand Stout,2021 vintage bottle 79\n\nHoly Fucking Shit KB...,0.553724
3,4,Kentucky Brunch Brand Stout,Celebrating my buddy Rug with his 1000th beer ...,0.529376
4,5,Kentucky Brunch Brand Stout,Thick and syrupy pour mocha head Aroma is booz...,0.562470
...,...,...,...,...
2508,2509,Coolship Red,Very interesting and complex ale First this is...,0.546140
2509,2510,Coolship Red,Appearance The beer was pinkish red is color ...,0.422693
2510,2511,Coolship Red,Look beautiful opaque garnet red Soft white he...,0.591636
2511,2512,Coolship Red,Caged and corked bottle dated dec32020 and ope...,0.637291


In [25]:
spacygroups = scores[scores['spacy_score'] != 0].groupby(by='product_name').mean()


In [26]:
spacygroups

,Unnamed: 0,spacy_score
product_name,,
4th Anniversary,975.0,0.281752
A Deal With The Devil - Double Oak-Aged,197.5,0.446775
A Deal With The Devil - Triple Oak-Aged,175.5,0.323392
Abner,395.0,0.412867
Abrasive Ale,1745.5,0.431811
...,...,...
XTRA DUBL Benthic,1792.5,0.392970
Yellow Bus,2086.5,0.431738
Zenne Y Frontera,120.0,0.433651


In [29]:
def no_attrs(s):
    attrs = ['chocolate', "dark" , "sweet" , "bourbon" ,"coffee"]
    s = s.lower()
    s = re.sub(r'[^A-Za-z0-9 ]+', '', s)
    s = s.split()
    for attr in attrs:
        if attr in s:
            return True
    return False
  
def remove_stop(sent):
    new = []
    for w in sent:
        if w not in stop:
            new.append(w)
    return new



In [34]:
stop = stopwords.words('english')
stop += ['like','dont','would','rt', 'im', 'today', 'http', 'amp', 'https', 'let']

In [36]:
def get_window(attr, sent):
    if attr not in sent:
        return None
    attr_idx = sent.index(attr)
    i, j = attr_idx - 2, attr_idx + 2
    if i < 0:
        i = 0
    if j > len(sent):
        j = len(sent)
    return sent[i:j]


NameError: ignored